We can use this dataset about HackerNews to query with BigQuery word frequency about articles about topics. I was thinking about how SolarWinds appeared in the past compared to how it is appearing now (note we will need a new dataset to make the second query).  Let's find out using the BigQuery Hacker News dataset! I'll use the BigQuery Python client library to query the stories table of the hacker_news dataset where title contains solarWinds or SolarWinds.

Before I begin, note that in total size, the Hacker News BigQuery dataset is 14GB which is larger than most Kaggle Datasets (the file upload limit is 10GB uncompressed). It's not big in BigQuery terms, but it's still a good idea for me to avoid using SELECT * statements so that I don't query more than necessary against my 5TB monthly free quota.

Let's get started!

In [ ]:
from google.cloud import bigquery
import pandas as pd



client = bigquery.Client()

# Using WHERE reduces the amount of data scanned / quota used
query = """
SELECT title, time_ts
FROM `bigquery-public-data.hacker_news.stories`
WHERE REGEXP_CONTAINS(title, r"(s|S)olarWinds")
ORDER BY time
"""

query_job = client.query(query)

iterator = query_job.result(timeout=30)
rows = list(iterator)

# Transform the rows into a nice pandas dataframe
headlines = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

# Look at the first 10 headlines
headlines.head(10)

Very cool! It was super quick and easy to pull down this data. A next step would be to **compare mentions of SolarWinds now**...but that will be for later. 

Alright, now that the headlines in a nice dataframe, I think the natural next step is to create a word cloud. Let's go ahead and do that and see what stands out.

In [ ]:
import wordcloud
import matplotlib.pyplot as plt

words = ' '.join(headlines.title).lower()
cloud = wordcloud.WordCloud(background_color='black',
                            max_font_size=200,
                            width=1600,
                            height=800,
                            max_words=300,
                            relative_scaling=.5).generate(words)
plt.figure(figsize=(20,10))
plt.axis('off')
plt.savefig('SolarWinds-hackernews.png')
plt.imshow(cloud);

Great! Let me know what you think about using the BigQuery Python client libraries to query BigQuery datasets. If you want to try it out yourself, I recommend a few resources:

Sohier's "Getting Started" kernel will show you the basics plus he shares more about the BigQuery API here.

Check out these tips for managing resources in BigQuery (so you don't exhaust your 5TB too quickly).

See all BigQuery datasets that are accessible on Kaggle via Kernels here: https://www.kaggle.com/datasets?filetype=bigQuery